In [1]:
import requests 
import json
import time
import pandas as pd
import numpy as np
import math
import os
from sklearn.model_selection import train_test_split 

In [2]:
#Importing dataset
df = pd.read_csv("cleaned_review_v3.csv")
df.head (10)

,recommendationid,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,appid,steamid,num_games_owned,num_reviews,playtime_forever,playtime_last_two_weeks,playtime_at_review,last_played
0,79670406,A unique project that combines a number of str...,2020-11-22 17:28:32,2020-11-22 17:31:09,True,43,0,0.704378,0,True,False,False,1018130,76561198085502382,2870,154,95,0,90.0,2020-11-22 17:30:13
1,79617127,"cool air hockey, with the ability to build bar...",2020-11-21 17:44:34,2020-11-21 17:44:34,True,42,0,0.660510,0,True,False,False,1018130,76561198029816126,900,26,70,0,70.0,2020-11-20 21:31:16
2,79674307,"Castle Break has cool retro style, awesome mus...",2020-11-22 18:56:48,2020-11-22 18:56:48,True,36,0,0.626081,0,True,False,False,1018130,76561198036172352,312,127,166,0,112.0,2020-11-22 19:58:24
3,76792205,This game felt like a really neat concept exec...,2020-09-30 02:19:43,2020-09-30 02:21:58,False,7,0,0.599947,0,True,False,False,1018130,76561198152633172,0,3,38,0,38.0,2020-09-30 02:14:08
4,73910081,- not a bad game ...\n-arcade indie platformer...,2020-08-07 11:41:28,2020-08-07 11:41:28,True,17,1,0.593609,0,True,False,False,1018130,76561198177828002,747,26,230,0,141.0,2020-08-07 15:40:44
5,64897863,Very cool arkanoid with its own chips. In addi...,2020-03-12 06:28:55,2020-03-12 06:28:55,True,8,0,0.549300,0,True,False,False,1018130,76561198068809540,4317,23,120,0,120.0,2020-03-09 09:04:12
6,79470416,"Very interesting game, hard to come off\nYou n...",2020-11-18 17:35:57,2020-11-18 17:35:57,True,3,0,0.548694,0,True,False,False,1018130,76561199032976456,301,30,230,0,230.0,2020-11-18 17:35:05
7,49583478,Its a great hardcore oldschool arcaniod-style ...,2019-03-16 01:29:05,2019-03-16 17:15:07,True,4,0,0.546830,0,True,False,False,1018130,76561198036949292,0,13,477,0,313.0,2020-06-26 18:00:56
8,79371729,"Quite an interesting indie game, in a retro ar...",2020-11-16 19:40:30,2020-11-16 19:40:30,True,5,2,0.544207,0,True,False,False,1018130,76561198130345311,2968,168,90,0,90.0,2020-11-16 19:34:12
9,79576053,No regret when buying.. 10/10 Recommended if u...,2020-11-20 22:40:00,2020-11-20 22:40:00,True,3,1,0.543568,0,True,False,False,1018130,76561198281743274,112,2,15,0,11.0,2021-04-07 10:49:09


In [3]:
# Drop the unnecessary columns
df = pd.read_csv("cleaned_review_v3.csv")
df = df.drop(['votes_funny','timestamp_created','timestamp_updated','comment_count', 'steam_purchase', 'received_for_free', 'written_during_early_access',
             'num_games_owned', 'num_reviews', 'playtime_forever' , 'playtime_last_two_weeks', 'playtime_at_review', 'last_played'], axis = 1 )
df.head(10)


,recommendationid,review,voted_up,votes_up,weighted_vote_score,appid,steamid
0,79670406,A unique project that combines a number of str...,True,43,0.704378,1018130,76561198085502382
1,79617127,"cool air hockey, with the ability to build bar...",True,42,0.660510,1018130,76561198029816126
2,79674307,"Castle Break has cool retro style, awesome mus...",True,36,0.626081,1018130,76561198036172352
3,76792205,This game felt like a really neat concept exec...,False,7,0.599947,1018130,76561198152633172
4,73910081,- not a bad game ...\n-arcade indie platformer...,True,17,0.593609,1018130,76561198177828002
5,64897863,Very cool arkanoid with its own chips. In addi...,True,8,0.549300,1018130,76561198068809540
6,79470416,"Very interesting game, hard to come off\nYou n...",True,3,0.548694,1018130,76561199032976456
7,49583478,Its a great hardcore oldschool arcaniod-style ...,True,4,0.546830,1018130,76561198036949292
8,79371729,"Quite an interesting indie game, in a retro ar...",True,5,0.544207,1018130,76561198130345311
9,79576053,No regret when buying.. 10/10 Recommended if u...,True,3,0.543568,1018130,76561198281743274


In [4]:
# Convert the values of "voted_up" column into int (True = 1, False = 0)
df['voted_up'] = df['voted_up'].astype(int)
df.head(10)

,recommendationid,review,voted_up,votes_up,weighted_vote_score,appid,steamid
0,79670406,A unique project that combines a number of str...,1,43,0.704378,1018130,76561198085502382
1,79617127,"cool air hockey, with the ability to build bar...",1,42,0.660510,1018130,76561198029816126
2,79674307,"Castle Break has cool retro style, awesome mus...",1,36,0.626081,1018130,76561198036172352
3,76792205,This game felt like a really neat concept exec...,0,7,0.599947,1018130,76561198152633172
4,73910081,- not a bad game ...\n-arcade indie platformer...,1,17,0.593609,1018130,76561198177828002
5,64897863,Very cool arkanoid with its own chips. In addi...,1,8,0.549300,1018130,76561198068809540
6,79470416,"Very interesting game, hard to come off\nYou n...",1,3,0.548694,1018130,76561199032976456
7,49583478,Its a great hardcore oldschool arcaniod-style ...,1,4,0.546830,1018130,76561198036949292
8,79371729,"Quite an interesting indie game, in a retro ar...",1,5,0.544207,1018130,76561198130345311
9,79576053,No regret when buying.. 10/10 Recommended if u...,1,3,0.543568,1018130,76561198281743274


In [5]:
#Checking the number of positive reviews and negative reviews
# 1: positive
# 0: negative
df['voted_up'].value_counts()

voted_up
1    145957
0     56851
Name: count, dtype: int64

In [6]:
#Remove some of the positive samples to balance out the dataset (preventing bias)
voted_up_true = df[df['voted_up'] == 1]
num_rows_to_remove = 75000
rows_to_remove = voted_up_true .sample(n=num_rows_to_remove, random_state=42)
df = df.drop(rows_to_remove.index)
df['voted_up'].value_counts()

voted_up
1    70957
0    56851
Name: count, dtype: int64

In [7]:
# Split the dataset into train set and test set
# Features: df['review'] (user's review)
# Label: df['voted_up'] (positive or negative )
X = df['review']
y = df['voted_up']

# 80% into train size,  20% into test size
X_train, X_test, y_train, y_test = train_test_split(X,y , 
                                   random_state=42,  
                                   test_size=0.2,  
                                   shuffle=True) 



In [8]:
# Checking for the train set
print(X_train.head(10))
y_train.value_counts()

188392    One of the best games of all time, graphics ar...
158442    For the price it is now, it's well worth playi...
115284    decent game, but a bit overpriced.  i played o...
10979     As you can see from my ridiculous amount of ho...
186807                                         advoid this!
59981     Guess who's back, back again to overanalyze th...
78324     A game for those who love animals. Greenpeace ...
12275     I bought this set and i cant find it when i tr...
103789    With the reviews for the game, I had low hopes...
63322     I've been playing Don't Nod's games more or le...
Name: review, dtype: object


voted_up
1    56703
0    45543
Name: count, dtype: int64

In [9]:
# Checking for the test set
print(X_test.head(10))
y_test.value_counts()

130016    Title: Marvel's Midnight Suns: A Heroic Fusion...
170004    hi guys. new player, i bought the add on.\nsho...
107724    I would not recommend 'Tulpa'. \n\nAs a game, ...
201102    Very interesting at first but the later levels...
148242    If you’re having trouble getting this game to ...
125597    This really annoys me about this game; I alrea...
62712     Menzoberranzan is a fairly easy game and has s...
184479    I have liked what I have played so far. There ...
164458                                         Recommended!
99095     A great shooter with an interesting graphical ...
Name: review, dtype: object


voted_up
1    14254
0    11308
Name: count, dtype: int64

## Applying Models

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import  TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
import time


In [11]:
# Using Sklearn CountVectorizer() with stop_words='english' to transform the texts into vectors to use in further text analysis
start_time = time.time() 
vect = CountVectorizer(stop_words='english')

#Fit into the dataset
vect.fit(X_train.values.astype('U'),y_train)

# Transform into vectorizing form
X_train_transformed= vect.transform(X_train.values.astype('U'))
X_test_transformed= vect.transform(X_test.values.astype('U'))
end_time = time.time()

vectorizing_time = end_time - start_time
# Check the vocabulary
vect.vocabulary_


{'best': 15085,
 'games': 42492,
 'time': 103633,
 'graphics': 44882,
 'breathtaking': 17422,
 'better': 15157,
 'gta': 45575,
 'vii': 110418,
 'storyline': 98597,
 'game': 42239,
 'recommended': 85294,
 'misclick': 63580,
 'price': 81588,
 'worth': 113810,
 'playing': 79579,
 'marble': 61017,
 'madness': 60204,
 'meets': 62057,
 'hamsterball': 46435,
 'remember': 86377,
 'ps3': 82558,
 'just': 54759,
 'fun': 41639,
 'racing': 83958,
 'played': 79535,
 'friends': 41194,
 'decent': 27924,
 'single': 94091,
 'player': 79539,
 'campaign': 19104,
 'mode': 64102,
 'bit': 15594,
 'overpriced': 76007,
 'completion': 23502,
 'theres': 102886,
 'going': 44277,
 '10': 186,
 'id': 49899,
 'wait': 111386,
 'pick': 78744,
 'sale': 89964,
 '00': 0,
 'unless': 108059,
 'really': 84817,
 'like': 58311,
 'type': 106432,
 'personally': 78263,
 'expected': 37081,
 'ridiculous': 88095,
 'hours': 49039,
 'enjoyed': 35103,
 've': 109782,
 'bothered': 16967,
 'achievement': 7025,
 'steam': 98052,
 'review': 

In [12]:
model_info = {}
def add_to_dict (name, f1, acc, recall, preci, runtime):
    model_info[name] = {'f1_score': f1, "accuracy" : acc, 'recall': recall, 'precision_score': preci, 'run_time': runtime}



## MultinomialNB

In [13]:
mnb = MultinomialNB()
start_time = time.time() 
mnb.fit(X_train_transformed, y_train)
y_pred_class = mnb.predict(X_test_transformed)
end_time = time.time()
run_time = end_time - start_time + vectorizing_time

f1 =  f1_score(y_test, y_pred_class.astype(int))
acc = accuracy_score (y_test, y_pred_class.astype(int))
rec = recall_score (y_test, y_pred_class.astype(int))
preci = precision_score(y_test, y_pred_class.astype(int))

add_to_dict ('MultinomialNB', f1, acc, rec, preci, run_time )


## Logistic Regression

In [14]:
lr=LogisticRegression(max_iter= 200000)

start_time = time.time() 
lr.fit(X_train_transformed, y_train)
y_pred = lr.predict(X_test_transformed)
end_time = time.time() 
run_time = end_time - start_time + vectorizing_time

f1 =  f1_score(y_test, y_pred.astype(int))
acc = accuracy_score (y_test, y_pred.astype(int))
rec = recall_score (y_test, y_pred.astype(int))
preci = precision_score(y_test, y_pred.astype(int))

add_to_dict ('Logistic_Regression', f1, acc, rec, preci, run_time )

## Decision Tree

In [15]:
dtc = DecisionTreeClassifier()

start_time = time.time() 
dtc.fit(X_train_transformed, y_train)
y_pred = dtc.predict(X_test_transformed)
end_time = time.time() 
run_time = end_time - start_time + vectorizing_time

f1 =  f1_score(y_test, y_pred.astype(int))
acc = accuracy_score (y_test, y_pred.astype(int))
rec = recall_score (y_test, y_pred.astype(int))
preci = precision_score(y_test, y_pred.astype(int))

add_to_dict ('Decision_Tree', f1, acc, rec, preci, run_time )

## TF-IDF & PIPELINE

### DecisionTree

In [16]:
pipeline = Pipeline([
    ('vectorizing', CountVectorizer(stop_words='english')),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', DecisionTreeClassifier()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])


In [17]:

start_time = time.time()
pipeline.fit(X_train.values.astype('U'),y_train)
predictions = pipeline.predict(X_test.values.astype('U'))
end_time = time.time()
run_time = end_time - start_time 

f1 =  f1_score(y_test, predictions .astype(int))
acc = accuracy_score (y_test, predictions .astype(int))
rec = recall_score(y_test, predictions .astype(int))
preci = precision_score(y_test, predictions .astype(int))


add_to_dict ('Pipeline_DecisionTree', f1, acc, rec, preci, run_time )

### LogisticRegression

In [18]:
pipeline = Pipeline([
    ('vectorizing', CountVectorizer(stop_words='english')),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', LogisticRegression(max_iter= 200000)),  # train on TF-IDF vectors w/ Naive Bayes classifier
])



In [19]:
start_time = time.time()
pipeline.fit(X_train.values.astype('U'),y_train)
predictions = pipeline.predict(X_test.values.astype('U'))
end_time = time.time()
run_time = end_time - start_time

f1 =  f1_score(y_test, predictions .astype(int))
acc = accuracy_score (y_test, predictions .astype(int))
rec = recall_score (y_test, predictions .astype(int))
preci = precision_score(y_test, predictions .astype(int))


add_to_dict ('Pipeline_LogisticRegression', f1, acc, rec, preci, run_time )

## MultinomialNB

In [20]:
pipeline = Pipeline([
    ('vectorizing', CountVectorizer(stop_words='english')),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

pipeline.fit(X_train.values.astype('U'),y_train.values.astype('U'))

Pipeline(steps=[('vectorizing', CountVectorizer(stop_words='english')),
                ('tfidf', TfidfTransformer()),
                ('classifier', MultinomialNB())])

In [21]:
start_time = time.time() 
pipeline.fit(X_train.values.astype('U'),y_train)
predictions = pipeline.predict(X_test.values.astype('U'))
end_time = time.time()
run_time = end_time - start_time

f1 =  f1_score(y_test, predictions .astype(int))
acc = accuracy_score (y_test, predictions .astype(int))
rec = recall_score (y_test, predictions .astype(int))
preci = precision_score(y_test, predictions .astype(int))


add_to_dict ('Pipeline_MutinomialNB', f1, acc, rec, preci, run_time )

In [22]:
def round_up_values(d):
    for key, value in d.items():
        if isinstance(value, dict):
            round_up_values(value)  # Recursively process nested dictionaries
        elif isinstance(value, (int, float)):
            d[key] = round(value, 3)  # Round to 3 decimal places

round_up_values(model_info)

model_info

{'MultinomialNB': {'f1_score': 0.839,
  'accuracy': 0.822,
  'recall': 0.829,
  'precision_score': 0.848,
  'run_time': 21.232},
 'Logistic_Regression': {'f1_score': 0.854,
  'accuracy': 0.832,
  'recall': 0.883,
  'precision_score': 0.828,
  'run_time': 39.163},
 'Decision_Tree': {'f1_score': 0.742,
  'accuracy': 0.712,
  'recall': 0.742,
  'precision_score': 0.742,
  'run_time': 272.382},
 'Pipeline_DecisionTree': {'f1_score': 0.741,
  'accuracy': 0.71,
  'recall': 0.742,
  'precision_score': 0.739,
  'run_time': 270.355},
 'Pipeline_LogisticRegression': {'f1_score': 0.864,
  'accuracy': 0.845,
  'recall': 0.885,
  'precision_score': 0.844,
  'run_time': 15.738},
 'Pipeline_MutinomialNB': {'f1_score': 0.848,
  'accuracy': 0.822,
  'recall': 0.887,
  'precision_score': 0.812,
  'run_time': 12.149}}

In [23]:
from tabulate import tabulate
table_data = []
for model, metrics in model_info.items():
    row = [model] + [round(value, 3) for value in metrics.values()]
    table_data.append(row)

# Define column headers
headers = list(model_info['MultinomialNB'].keys())

# Generate and print the table using tabulate
table = tabulate(table_data, headers=headers, tablefmt='grid')
print(table)

+-----------------------------+------------+------------+----------+-------------------+------------+
|                             |   f1_score |   accuracy |   recall |   precision_score |   run_time |
+=============================+============+============+==========+===================+============+
| MultinomialNB               |      0.839 |      0.822 |    0.829 |             0.848 |     21.232 |
+-----------------------------+------------+------------+----------+-------------------+------------+
| Logistic_Regression         |      0.854 |      0.832 |    0.883 |             0.828 |     39.163 |
+-----------------------------+------------+------------+----------+-------------------+------------+
| Decision_Tree               |      0.742 |      0.712 |    0.742 |             0.742 |    272.382 |
+-----------------------------+------------+------------+----------+-------------------+------------+
| Pipeline_DecisionTree       |      0.741 |      0.71  |    0.742 |             0